In [1]:
import math
import cv2
import numpy as np 
from PIL import Image
from matplotlib import pyplot as plt 
import sys, os
import random as rn 
import time
import scipy
import sklearn
from sklearn import svm
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [21]:
def read_img(path):
    imlist  = []
    labels = []


    c = 0
    im1 = os.listdir(path)
    for i in im1:
        if i != '.DS_Store':
            dire = os.path.join(path, i)
            im2 = os.listdir(dire)
            for j in im2:
                if j != '.DS_Store':
                    dir1 = os.path.join(dire, j)
                    im3 = os.listdir(dir1)
                    for k in im3:
                        if (k != '.DS_Store') and (k.endswith('jpg')):
                            imlist.append(os.path.join(dir1, k))
    imlist = np.array(imlist)
    for imfile in imlist:
        t = imfile.split('/')
        labels.append(t[8])
    
    labels = np.array(labels)
    print(labels)
    print(labels.shape)
    print(imlist.shape)
    return imlist, labels

In [4]:
def split_data(imlist, labels):
    
    DivingSide = []
    WalkFront = []
    GolfSwing = []
    Kicking = []
    Lifting = []
    RidingHorse = []
    Run = []
    SkateBoarding = []
    Swing = []

    DivingSide_label = []
    WalkFront_label = []
    GolfSwing_label = []
    Kicking_label = []
    Lifting_label = []
    RidingHorse_label = []
    Run_label = []
    SkateBoarding_label = []
    Swing_label = []




    for t in imlist:
        j = t.split('/')[-3]
        if j == 'Diving-Side':
            DivingSide.append(t)
            DivingSide_label.append('DivingSide')
            #print t
        if j == 'Golf-Swing-Back' or j == 'Golf-Swing-Front' or j == 'Golf-Swing-Side':
            GolfSwing.append(t)
            GolfSwing_label.append('GolfSwing')
        if j == 'Kicking-Side' or j == 'Kicking-Front':
            Kicking.append(t)
            Kicking_label.append('Kicking')
        if j == 'Lifting':
            Lifting.append(t)
            Lifting_label.append('Lifting')
        if j == 'Riding-Horse':
            RidingHorse.append(t)
            RidingHorse_label.append('RidingHorse')
        if j == 'Run-Side':
            Run.append(t)
            Run_label.append('Run')
        if j == 'SkateBoarding-Front':
            SkateBoarding.append(t)
            SkateBoarding_label.append('SkateBoarding')
        if j == 'Swing-Bench' or j == 'Swing-SideAngle':
            Swing.append(t)
            Swing_label.append('Swing')
        if j == 'Walk-Front':
            WalkFront.append(t)
            WalkFront_label.append('WalkFront')


    print (len(WalkFront_label)+ len(DivingSide_label)+  len(GolfSwing_label)+ len(Kicking_label)+ len(Lifting_label)+  len(RidingHorse_label)+  len(Run_label) +  len(Swing_label) +  len(SkateBoarding_label))
    return WalkFront, DivingSide, GolfSwing, Kicking, Lifting, RidingHorse, Run, Swing, SkateBoarding, WalkFront_label, DivingSide_label, GolfSwing_label, Kicking_label,  Lifting_label,   RidingHorse_label, Run_label , Swing_label, SkateBoarding_label




In [5]:
def image_process(image):
    I = Image.open(image)
    I1 = np.array(I)
    size = cv2.resize(I1, (300, 300), interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(size, cv2.COLOR_BGR2GRAY)

    return gray

In [6]:
def hog(image):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    # quantizing binvalues in (0...16)
    bins = np.int32(bin_n*ang/(2*np.pi))

    # Divide to 4 sub-squares
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)
    return hist



In [8]:
def get_features(WalkFront, DivingSide, GolfSwing, Kicking, Lifting, RidingHorse, Run, Swing, SkateBoarding):
    DivingSide_data = []
    WalkFront_data  = []
    GolfSwing_data  = []
    Kicking_data  = []
    Lifting_data  = []
    RidingHorse_data  = []
    Run_data  = []
    SkateBoarding_data  = []
    Swing_data  = []

    for i in range(len(DivingSide)):
        frame = image_process(DivingSide[i])
        data = hog(frame)
        DivingSide_data.append(data)
        print(1, i)
    print (np.shape(np.array(DivingSide_data)))

    for i in range(len(WalkFront)):
        frame = image_process(WalkFront[i])
        data = hog(frame)
        WalkFront_data.append(data)
        print(2, i)
    print (np.shape(np.array(WalkFront_data)))

    for i in range(len(GolfSwing)):
        frame = image_process(GolfSwing[i])
        data = hog(frame)
        GolfSwing_data.append(data)
        print(3, i)
    print(np.shape(np.array(GolfSwing_data)))

    for i in range(len(Kicking)):
        frame = image_process(Kicking[i])
        data = hog(frame)
        Kicking_data.append(data)
        print(4, i)
    print(np.shape(np.array(Kicking_data)))

    for i in range(len(Lifting)):
        frame = image_process(Lifting[i])
        data = hog(frame)
        Lifting_data.append(data)
        print(5, i)
    print (np.shape(np.array(Lifting_data)))

    for i in range(len(RidingHorse)):
        frame = image_process(RidingHorse[i])
        data = hog(frame)
        RidingHorse_data.append(data)
        print(6, i)
    print (np.shape(np.array(RidingHorse_data)))

    for i in range(len(Run)):
        frame = image_process(Run[i])
        data = hog(frame)
        Run_data.append(data)
        print(7, i)
    print(np.shape(np.array(Run_data)))

    for i in range(len(SkateBoarding)):
        frame = image_process(SkateBoarding[i])
        data = hog(frame)
        SkateBoarding_data.append(data)
        print(8, i)
    print(np.shape(np.array(SkateBoarding_data)))

    for i in range(len(Swing)):
        frame = image_process(Swing[i])
        data = hog(frame)
        Swing_data.append(data)
        print(9, i)
    print(np.shape(np.array(Swing_data)))

    return WalkFront_data, DivingSide_data, GolfSwing_data, Kicking_data,  Lifting_data,   RidingHorse_data, Run_data , Swing_data, SkateBoarding_data

In [9]:
def Leave_One_Out(i, subset_size, data, label):
    data_test = []
    data_train = []
    label_test = []
    label_train = []
    data_test = data[i*subset_size:][:subset_size]
    data_train = data[:i*subset_size] + data[(i+1)*subset_size:]
    label_test = label[i*subset_size:][:subset_size]
    label_train = label[:i*subset_size] + label[(i+1)*subset_size:]
    return data_test, data_train, label_test, label_train


In [11]:
def eval(arr):
    TP = []
    TN = []
    FP = []
    FN = []
    sensitivity = []
    specificity = []
    for i in range(9):
        for j in range(9):
            if i == j:
                TP.append(arr[i][j])
    sum_cols = arr.sum(axis = 1)
    sum_rows = arr.sum(axis = 0)
    for i in range(9):
        FN.append(sum_cols[i] - TP[i])
        FP.append(sum_rows[i] - TP[i])
        TN.append(arr.sum()-sum_rows[i] - sum_cols[i])
    print(TP, TN, FP, FN)
    for i in range(9):
        sense = (TP[i]/(TP[i] + FN[i]))
        spec = (TN[i]/(FP[i] + TN[i]))
        sensitivity.append(sense)
        specificity.append(spec)
    return sensitivity, specificity

In [12]:
def Evaluation_SVM(data_train, data_test, label_train, label_test, i):
    clf = svm.SVC( kernel = 'linear', C = 1, gamma = 0.0000001)
    clf.fit(data_train, label_train)
    predicted = clf.predict(data_test)
    error = (label_test == predicted).mean()
    label = [ 'DivingSide', 'WalkFront','GolfSwing', 'Kicking', 'Lifting', 'RidingHorse', 'Run', 'SkateBoarding' , 'Swing']
    arr = confusion_matrix(label_test, predicted, label)
    print (arr)
    print ('for', i, 'iteration the accuracy of svm is %.2f %%' %(error*100))
    return error, arr

In [13]:
def train_test(WalkFront_data, DivingSide_data, GolfSwing_data, Kicking_data,  Lifting_data,   RidingHorse_data, Run_data , Swing_data, SkateBoarding_data, WalkFront_label, DivingSide_label, GolfSwing_label, Kicking_label,  Lifting_label,   RidingHorse_label, Run_label , Swing_label, SkateBoarding_label):

    num_folds = 5
    label = [ 'DivingSide', 'WalkFront','GolfSwing', 'Kicking', 'Lifting', 'RidingHorse', 'Run', 'SkateBoarding' , 'Swing']
    
    DivingSide_train = []
    WalkFront_train = []
    GolfSwing_train = []
    Kicking_train = []
    Lifting_train = []
    RidingHorse_train = []
    Run_train = []
    SkateBoarding_train = []
    Swing_train = []

    DivingSide_test = []
    WalkFront_test = []
    GolfSwing_test = []
    Kicking_test = []
    Lifting_test = []
    RidingHorse_test = []
    Run_test = []
    SkateBoarding_test = []
    Swing_test = []

    DivingSide_label_train = []
    WalkFront_label_train = []
    GolfSwing_label_train = []
    Kicking_label_train = []
    Lifting_label_train = []
    RidingHorse_label_train = []
    Run_label_train = []
    SkateBoarding_label_train = []
    Swing_label_train = []

    DivingSide_label_test = []
    WalkFront_label_test = []
    GolfSwing_label_test = []
    Kicking_label_test = []
    Lifting_label_test = []
    RidingHorse_label_test = []
    Run_label_test = []
    SkateBoarding_label_test = []
    Swing_label_test = []

    training_data = []
    test_data = []
    training_label = []
    test_label = []
    err = []
    sensitivity = []
    specificity = []

    
    for i in range(num_folds):
        subset_size = int(len(DivingSide_data)/num_folds)
        DivingSide_test, DivingSide_train, 	DivingSide_label_test, DivingSide_label_train = Leave_One_Out(i, subset_size, DivingSide_data, DivingSide_label)

        subset_size = int(len(WalkFront_data)/num_folds)
        WalkFront_test, WalkFront_train, 	WalkFront_label_test, WalkFront_label_train = Leave_One_Out(i, subset_size, WalkFront_data, WalkFront_label)

        subset_size = int(len(GolfSwing_data)/num_folds)
        GolfSwing_test, GolfSwing_train, 	GolfSwing_label_test, GolfSwing_label_train = Leave_One_Out(i, subset_size, GolfSwing_data, GolfSwing_label)

        subset_size = int(len(Kicking_data)/num_folds)
        Kicking_test, Kicking_train, Kicking_label_test, Kicking_label_train = Leave_One_Out(i, subset_size, Kicking_data, Kicking_label)

        subset_size = int(len(Lifting_data)/num_folds)
        Lifting_test, Lifting_train, Lifting_label_test, Lifting_label_train = Leave_One_Out(i, subset_size, Lifting_data, Lifting_label)

        subset_size = int(len(RidingHorse_data)/num_folds)
        RidingHorse_test, RidingHorse_train, RidingHorse_label_test, RidingHorse_label_train = Leave_One_Out(i, subset_size, RidingHorse_data, RidingHorse_label)

        subset_size = int(len(Run_data)/num_folds)
        Run_test, Run_train, Run_label_test, Run_label_train = Leave_One_Out(i, subset_size, Run_data, Run_label)

        subset_size = int(len(SkateBoarding_data)/num_folds)
        SkateBoarding_test, SkateBoarding_train, SkateBoarding_label_test, SkateBoarding_label_train = Leave_One_Out(i, subset_size, SkateBoarding_data, SkateBoarding_label)

        subset_size = int(len(Swing_data)/num_folds)
        Swing_test, Swing_train, Swing_label_test, Swing_label_train = Leave_One_Out(i, subset_size, Swing_data, Swing_label)

        print('Splitting each class into test and train complete !', i)

        training_data = DivingSide_train + WalkFront_train + GolfSwing_train + Kicking_train + Lifting_train + RidingHorse_train + Run_train + SkateBoarding_train + Swing_train
        test_data = DivingSide_test + WalkFront_test + GolfSwing_test + Kicking_test + Lifting_test + RidingHorse_test + Run_test + SkateBoarding_test + Swing_test
        training_label = DivingSide_label_train + WalkFront_label_train + GolfSwing_label_train + Kicking_label_train + Lifting_label_train + RidingHorse_label_train +	Run_label_train + SkateBoarding_label_train + Swing_label_train
        test_label = DivingSide_label_test + WalkFront_label_test + GolfSwing_label_test + Kicking_label_test + Lifting_label_test + RidingHorse_label_test + Run_label_test + SkateBoarding_label_test + Swing_label_test

        error, arr = Evaluation_SVM(training_data, test_data, training_label, test_label, i)
        err.append(error)
        print(np.shape(np.array(test_data)), np.shape(np.array(training_data)))
        sensitivity, specificity =  eval(arr)
    accuracy = sum(err)/len(err)
    print('Overall accuracy is: %.2f %%' %(accuracy*100))
    for i in range(9):
        print('Sensitivity of', label[i], 'is' , sensitivity[i])
        print('Specificity of', label[i], 'is', specificity[i])

In [17]:
def res():
    imlist = [] 
    labels = []

    DivingSide = []
    WalkFront = []
    GolfSwing = []
    Kicking = []
    Lifting = []
    RidingHorse = []
    Run = []
    SkateBoarding = []
    Swing = []

    DivingSide_label = []
    WalkFront_label = []
    GolfSwing_label = []
    Kicking_label = []
    Lifting_label = []
    RidingHorse_label = []
    Run_label = []
    SkateBoarding_label = []
    Swing_label = []

    DivingSide_data = []
    WalkFront_data  = []
    GolfSwing_data  = []
    Kicking_data  = []
    Lifting_data  = []
    RidingHorse_data  = []
    Run_data  = []
    SkateBoarding_data  = []
    Swing_data  = []

    imlist, labels = read_img("/home/harshithapv/srikar cvpa3/ucf_sports_actions/ucf action")
    WalkFront, DivingSide, GolfSwing, Kicking, Lifting, RidingHorse, Run, Swing, SkateBoarding, WalkFront_label, DivingSide_label, GolfSwing_label, Kicking_label,  Lifting_label,   RidingHorse_label, Run_label , Swing_label, SkateBoarding_label = split_data(imlist, labels)
    WalkFront_data, DivingSide_data, GolfSwing_data, Kicking_data,  Lifting_data,   RidingHorse_data, Run_data , Swing_data, SkateBoarding_data = get_features(WalkFront, DivingSide, GolfSwing, Kicking, Lifting, RidingHorse, Run, Swing, SkateBoarding)
    
    train_test(WalkFront_data, DivingSide_data, GolfSwing_data, Kicking_data,  Lifting_data,   RidingHorse_data, Run_data , Swing_data, SkateBoarding_data, WalkFront_label, DivingSide_label, GolfSwing_label, Kicking_label,  Lifting_label,   RidingHorse_label, Run_label , Swing_label, SkateBoarding_label)


In [22]:
bin_n = 16
res()

['6063-21_70057.jpg' '6063-21_70070.jpg' '6063-21_70066.jpg' ...,
 '5863-29_70035.jpg' '5863-29_70032.jpg' '5863-29_70034.jpg']
(9220,)
(9220,)
9220
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1

2 615
2 616
2 617
2 618
2 619
2 620
2 621
2 622
2 623
2 624
2 625
2 626
2 627
2 628
2 629
2 630
2 631
2 632
2 633
2 634
2 635
2 636
2 637
2 638
2 639
2 640
2 641
2 642
2 643
2 644
2 645
2 646
2 647
2 648
2 649
2 650
2 651
2 652
2 653
2 654
2 655
2 656
2 657
2 658
2 659
2 660
2 661
2 662
2 663
2 664
2 665
2 666
2 667
2 668
2 669
2 670
2 671
2 672
2 673
2 674
2 675
2 676
2 677
2 678
2 679
2 680
2 681
2 682
2 683
2 684
2 685
2 686
2 687
2 688
2 689
2 690
2 691
2 692
2 693
2 694
2 695
2 696
2 697
2 698
2 699
2 700
2 701
2 702
2 703
2 704
2 705
2 706
2 707
2 708
2 709
2 710
2 711
2 712
2 713
2 714
2 715
2 716
2 717
2 718
2 719
2 720
2 721
2 722
2 723
2 724
2 725
2 726
2 727
2 728
2 729
2 730
2 731
2 732
2 733
2 734
2 735
2 736
2 737
2 738
2 739
2 740
2 741
2 742
2 743
2 744
2 745
2 746
2 747
2 748
2 749
2 750
2 751
2 752
2 753
2 754
2 755
2 756
2 757
2 758
2 759
2 760
2 761
2 762
2 763
2 764
2 765
2 766
2 767
2 768
2 769
2 770
2 771
2 772
2 773
2 774
2 775
2 776
2 777
2 778
2 779
2 780
2 78

2 1842
2 1843
2 1844
2 1845
2 1846
2 1847
2 1848
2 1849
2 1850
2 1851
2 1852
2 1853
2 1854
2 1855
2 1856
2 1857
2 1858
2 1859
2 1860
2 1861
2 1862
2 1863
2 1864
2 1865
2 1866
2 1867
2 1868
2 1869
2 1870
2 1871
2 1872
2 1873
2 1874
2 1875
2 1876
2 1877
2 1878
2 1879
2 1880
2 1881
2 1882
2 1883
2 1884
2 1885
2 1886
2 1887
2 1888
2 1889
2 1890
2 1891
2 1892
2 1893
2 1894
2 1895
2 1896
2 1897
2 1898
2 1899
2 1900
2 1901
2 1902
2 1903
2 1904
2 1905
2 1906
2 1907
2 1908
2 1909
2 1910
2 1911
2 1912
2 1913
2 1914
2 1915
2 1916
2 1917
2 1918
2 1919
2 1920
2 1921
2 1922
2 1923
2 1924
2 1925
2 1926
2 1927
2 1928
2 1929
2 1930
2 1931
2 1932
2 1933
2 1934
2 1935
2 1936
2 1937
2 1938
2 1939
2 1940
2 1941
2 1942
2 1943
2 1944
2 1945
2 1946
2 1947
2 1948
2 1949
2 1950
2 1951
2 1952
2 1953
2 1954
2 1955
2 1956
2 1957
2 1958
2 1959
2 1960
2 1961
2 1962
2 1963
2 1964
2 1965
2 1966
2 1967
2 1968
2 1969
2 1970
2 1971
2 1972
2 1973
2 1974
2 1975
2 1976
2 1977
2 1978
2 1979
2 1980
2 1981
2 1982
2 1983
2 1984

4 283
4 284
4 285
4 286
4 287
4 288
4 289
4 290
4 291
4 292
4 293
4 294
4 295
4 296
4 297
4 298
4 299
4 300
4 301
4 302
4 303
4 304
4 305
4 306
4 307
4 308
4 309
4 310
4 311
4 312
4 313
4 314
4 315
4 316
4 317
4 318
4 319
4 320
4 321
4 322
4 323
4 324
4 325
4 326
4 327
4 328
4 329
4 330
4 331
4 332
4 333
4 334
4 335
4 336
4 337
4 338
4 339
4 340
4 341
4 342
4 343
4 344
4 345
4 346
4 347
4 348
4 349
4 350
4 351
4 352
4 353
4 354
4 355
4 356
4 357
4 358
4 359
4 360
4 361
4 362
4 363
4 364
4 365
4 366
4 367
4 368
4 369
4 370
4 371
4 372
4 373
4 374
4 375
4 376
4 377
4 378
4 379
4 380
4 381
4 382
4 383
4 384
4 385
4 386
4 387
4 388
4 389
4 390
4 391
4 392
4 393
4 394
4 395
4 396
4 397
4 398
4 399
4 400
4 401
4 402
4 403
4 404
4 405
4 406
4 407
4 408
4 409
4 410
4 411
4 412
4 413
4 414
4 415
4 416
4 417
4 418
4 419
4 420
4 421
4 422
4 423
4 424
4 425
4 426
4 427
4 428
4 429
4 430
4 431
4 432
4 433
4 434
4 435
4 436
4 437
4 438
4 439
4 440
4 441
4 442
4 443
4 444
4 445
4 446
4 447
4 448
4 44

6 528
6 529
6 530
6 531
6 532
6 533
6 534
6 535
6 536
6 537
6 538
6 539
6 540
6 541
6 542
6 543
6 544
6 545
6 546
6 547
6 548
6 549
6 550
6 551
6 552
6 553
6 554
6 555
6 556
6 557
6 558
6 559
6 560
6 561
6 562
6 563
6 564
6 565
6 566
6 567
6 568
6 569
6 570
6 571
6 572
6 573
6 574
6 575
6 576
6 577
6 578
6 579
6 580
6 581
6 582
6 583
6 584
6 585
6 586
6 587
6 588
6 589
6 590
6 591
6 592
6 593
6 594
6 595
6 596
6 597
6 598
6 599
6 600
6 601
6 602
6 603
6 604
6 605
6 606
6 607
6 608
6 609
6 610
6 611
6 612
6 613
6 614
6 615
6 616
6 617
6 618
6 619
6 620
6 621
6 622
6 623
6 624
6 625
6 626
6 627
6 628
6 629
6 630
6 631
6 632
6 633
6 634
6 635
6 636
6 637
6 638
6 639
6 640
6 641
6 642
6 643
6 644
6 645
6 646
6 647
6 648
6 649
6 650
6 651
6 652
6 653
6 654
6 655
6 656
6 657
6 658
6 659
6 660
6 661
6 662
6 663
6 664
6 665
6 666
6 667
6 668
6 669
6 670
6 671
6 672
6 673
6 674
6 675
6 676
6 677
6 678
6 679
6 680
6 681
6 682
6 683
6 684
6 685
6 686
6 687
6 688
6 689
6 690
6 691
6 692
6 693
6 69

8 374
8 375
8 376
8 377
8 378
8 379
8 380
8 381
8 382
8 383
8 384
8 385
8 386
8 387
8 388
8 389
8 390
8 391
8 392
8 393
8 394
8 395
8 396
8 397
8 398
8 399
8 400
8 401
8 402
8 403
8 404
8 405
8 406
8 407
8 408
8 409
8 410
8 411
8 412
8 413
8 414
8 415
8 416
8 417
8 418
8 419
8 420
8 421
8 422
8 423
8 424
8 425
8 426
8 427
8 428
8 429
8 430
8 431
8 432
8 433
8 434
8 435
8 436
8 437
8 438
8 439
8 440
8 441
8 442
8 443
8 444
8 445
8 446
8 447
8 448
8 449
8 450
8 451
8 452
8 453
8 454
8 455
8 456
8 457
8 458
8 459
8 460
8 461
8 462
8 463
8 464
8 465
8 466
8 467
8 468
8 469
8 470
8 471
8 472
8 473
8 474
8 475
8 476
8 477
8 478
8 479
8 480
8 481
8 482
8 483
8 484
8 485
8 486
8 487
8 488
8 489
8 490
8 491
8 492
8 493
8 494
8 495
8 496
8 497
8 498
8 499
8 500
8 501
8 502
8 503
8 504
8 505
8 506
8 507
8 508
8 509
8 510
8 511
8 512
8 513
8 514
8 515
8 516
8 517
8 518
8 519
8 520
8 521
8 522
8 523
8 524
8 525
8 526
8 527
8 528
8 529
8 530
8 531
8 532
8 533
8 534
8 535
8 536
8 537
8 538
8 539
8 54

9 934
9 935
9 936
9 937
9 938
9 939
9 940
9 941
9 942
9 943
9 944
9 945
9 946
9 947
9 948
9 949
9 950
9 951
9 952
9 953
9 954
9 955
9 956
9 957
9 958
9 959
9 960
9 961
9 962
9 963
9 964
9 965
9 966
9 967
9 968
9 969
9 970
9 971
9 972
9 973
9 974
9 975
9 976
9 977
9 978
9 979
9 980
9 981
9 982
9 983
9 984
9 985
9 986
9 987
9 988
9 989
9 990
9 991
9 992
9 993
9 994
9 995
9 996
9 997
9 998
9 999
9 1000
9 1001
9 1002
9 1003
9 1004
9 1005
9 1006
9 1007
9 1008
9 1009
9 1010
9 1011
9 1012
9 1013
9 1014
9 1015
9 1016
9 1017
9 1018
9 1019
9 1020
9 1021
9 1022
9 1023
9 1024
9 1025
9 1026
9 1027
9 1028
9 1029
9 1030
9 1031
9 1032
9 1033
9 1034
9 1035
9 1036
9 1037
9 1038
9 1039
9 1040
9 1041
9 1042
9 1043
9 1044
9 1045
9 1046
9 1047
9 1048
9 1049
9 1050
9 1051
9 1052
9 1053
9 1054
9 1055
9 1056
9 1057
9 1058
9 1059
9 1060
9 1061
9 1062
9 1063
9 1064
9 1065
9 1066
9 1067
9 1068
9 1069
9 1070
9 1071
9 1072
9 1073
9 1074
9 1075
9 1076
9 1077
9 1078
9 1079
9 1080
9 1081
9 1082
9 1083
9 1084
9 1085
9 

[[143   0   0   5   0   1   0   3   2]
 [  0 133   1  41   0 101   0 101  60]
 [  0  60  24  14   0   0  46   0   0]
 [  6   1   0  39   0   0  14  16  15]
 [  0   0   0   0 140   0   0   0   0]
 [  0   5   0  27   0 101   2   0   9]
 [  3   1  54   0   0   1 110   0   0]
 [  0   0   0   1   0   2  20 143   2]
 [ 14   2   0   2   0  22   0   0 355]]
for 2 iteration the accuracy of svm is 64.50 %
(1842, 64) (7378, 64)
[143, 133, 24, 39, 140, 101, 110, 143, 355] [1522, 1203, 1619, 1622, 1562, 1470, 1481, 1411, 1004] [23, 69, 55, 90, 0, 127, 82, 120, 88] [11, 304, 120, 52, 0, 43, 59, 25, 40]
Done Splitting each class into test and train 3
[[138   1   0   3   0   0   1   0  11]
 [ 15 120  90  11   1 130   0  70   0]
 [ 47   0  90   0   0   0   7   0   0]
 [  4   1   2  84   0   0   0   0   0]
 [  0   0   0   0 140   0   0   0   0]
 [  0   1   1   0   0  84   4  24  30]
 [  3   5   0   4   3   1  88   0  65]
 [  0   0   0  55   0   2   1 110   0]
 [  4   0   0  93   0   3   0   0 295]]
for 